In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
!pip install pymorphy2

In [2]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline


In [6]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [7]:
train = pd.read_csv('../input/scan-classification-challange/df_train.csv')
train.sample(3)

In [3]:
train.info()

In [8]:
test = pd.read_csv('../input/scan-classification-challange/df_test.csv')
test.sample(3)

In [5]:
test.drop(['Unnamed: 0'], inplace=True, axis=1)

In [9]:
sample_submission = pd.read_csv('../input/scan-classification-challange/sample_submission.csv')
sample_submission

In [4]:
submission

In [7]:
Средняя длина, стандартное отклонение, макс.длина столбца text
lens = train.text.str.len()
lens.mean(), lens.std(), lens.max()

In [8]:
lens.hist();

In [9]:
pd.unique(train['class'])

In [10]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
#test['class'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

df = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, df.shape)

In [12]:
# Смотрим кол-во по каждому классу
train['class'].value_counts()

In [15]:
# создаем новый столбец, закодирует столбец с классами
df['encoded_cat'] = df['class'].astype('category').cat.codes
df.drop(['class'], inplace=True, axis=1)
df.sample(5)

## Stopworlds

In [16]:
# Посмотрим на состав имеющихся стоп-слов
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
print(stopwords.words("russian"))

In [48]:
# Теперь выведем все слова длиной менее 3-х символов и дополним список стоп-слов

stopworlds_new = set()
mas_stop = set()
for words in df['text']:
    for i in words.split():
        if len(i) <= 3:
            mas_stop.add(i)
print(mas_stop)

In [49]:
# Создаем новый список стоп-слов
stopworlds_new = set(stopwords_ru).union(mas_stop)

In [54]:
# Создадим функцию, где будем удалять удалять различные знаки пунктуации, цифры и прочие символы
# Также будем исключать стоп слова, приводить слова к нижнему регистру
#stemmer = SnowballStemmer("russian") 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
def lemmatize(doc):
    doc = doc.lower()
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
 #   if len(tokens) >=0 :
     #   tokens = [word for word in tokens if not word in worlds_freq]
    return ' '.join(tokens)

In [57]:
df['text_clean'] = df.text.apply(lambda x: lemmatize(x))

In [60]:
# Выведем список слов которые часто встречаются
words = list( df['text_clean'].values)

split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopworlds_new:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist

In [106]:
# Получим 10 часто встречающихся слов

from nltk import FreqDist
word_freq = FreqDist(allwords).most_common(10)
word_freq

In [89]:
# создадим список из 10 слов наиболее часто встречающихся
worlds_freq = ['который', 'сообщить', 'дело', 'заявить', 'россия', 'сказать', 'свой', 'рф', 'сообщать', 'также']
#world_freq = []
#for i in word_freq:
#    print(str(i[:1]).strip())
#    world_freq.append(i[:1])
#world_freq

In [103]:
#patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
def freq(doc):
    doc = doc.lower()
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
           # token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
        if len(tokens) >=0 :
            tokens = [word for word in tokens if not word in worlds_freq] # добавим исключение частовстречающихся слов
    return ' '.join(tokens)

In [104]:
df['text_clean_freq'] = df.text_clean.apply(lambda x: freq(x))
df.text_clean_freq

In [105]:
# world cloud 
# выводим облако слов, сразу применяем стоп слова, и слова к нижнему регистру
comment_words = ''
stopwords = set(STOPWORDS)
 
# iterate through the csv file
#for val in df['text']:
#    val = str(val)
#    tokens = val.split()
#for i in range(len(allwords)):
 #   tokens[i] = tokens[i].lower()
     
comment_words += " ".join(allwords)+" " 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
               # stopwords = stopwords,
                stopwords = stopworlds_new,
                min_font_size = 10).generate(comment_words)                     
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
#plt.show()

In [114]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)
y = train_data.encoded_cat.values     # наш таргет
X = train_data.drop(['text', 'text_clean', 'encoded_cat'], axis=1)
X_sub = df.drop(['text_clean', 'text_clean_freq', 'sample'], axis=1)

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

In [129]:
# split данных
text_train = df.text_clean_freq.iloc[X_train.index]
text_test = df.text_clean_freq.iloc[X_test.index]
text_sub = df.text_clean_freq.iloc[X_sub.index]

In [135]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [137]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df.text_clean_freq)

In [138]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [139]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

## FastText

In [12]:
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

import fasttext
import pymorphy2
from string import punctuation
punkt= [p for p in punctuation] + ["`", "``" ,"''", "'"]
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('russian')

lemmatizer = pymorphy2.MorphAnalyzer(lang='ru')

In [45]:
#title_train, title_test, y_train, y_test = train_test_split(titles_preprocessed, y, test_size=0.2, stratify=y, random_state=42)
ft_train, ft_test = train_test_split(train, random_state=42, test_size=0.2, stratify = train['class'])

In [46]:
def to_ft_label(s):
    return '__label__'+s.replace(',','_').replace(' ','_').replace('-','_')

labels_dict = {}
for g in train['class']:
    labels_dict[to_ft_label(g)] = g

In [47]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
train.iloc[:, 0] = train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
test.iloc[:, 1] = test.iloc[:, 1].apply(lambda x: ' '.join(simple_preprocess(x)))

In [48]:
col = ['class', 'text']

# train
train_for_ft = ft_train[col]
train_for_ft['class']=[to_ft_label(s) for s in train_for_ft['class']]

# test
test_for_ft = ft_test[col]
test_for_ft['class']=[to_ft_label(s) for s in test_for_ft['class']]

In [49]:
train_for_ft.to_csv('train_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")
test_for_ft.to_csv('test_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")

In [50]:
model = fasttext.train_supervised('train_for_ft.csv', lr = 0.9)

In [51]:
model.test('test_for_ft.csv')

In [52]:
def predict(test):
    return labels_dict[ model.predict(test['text'], k=1)[0][0] ]
test['predictions'] = test.apply(predict,axis=1)

test.head()

In [37]:
submission = pd.DataFrame({'id':range(0, len(test)),
                           'class':test['predictions'].values},
                          columns=['id', 'class'])
submission.to_csv('submission1.csv', index=False)
submission.head()

In [ ]:
sub_predict = model.predict([text_sub_sequences, X_sub])
sample_submission['class'] = sub_predict_nn2[:,0]
sample_submission.to_csv('submission.csv', index=False)